In [0]:
delta_table_path = "/dbfs/delta-lake-tables/bronze"

df = spark.read.format("delta").load(delta_table_path)

In [0]:
# df.show()

+--------+----------------+----------+-----------+-----------------------+--------------------+-------------------+-------------------+-----------+---------+
|ID_Contr|ID_Tipo_Contrato|ID_Cenario|Total_Horas|Quantidade_MW_Medio_Mes|Data_Inicio_Vigencia|  Data_Fim_Vigencia|ID_Contratos_Mensal|Preco_Final|ID_Agente|
+--------+----------------+----------+-----------+-----------------------+--------------------+-------------------+-------------------+-----------+---------+
|    7430|               2|         1|        744|                      0| 2021-02-18 00:00:00|2021-02-23 00:00:00|             245560|       NULL|      -10|
|   61190|               2|        47|        720|                      0| 2023-05-29 00:00:00|9999-12-31 00:00:00|           12247464|       NULL|      110|
|   61190|               2|        47|        672|                      0| 2023-05-29 00:00:00|9999-12-31 00:00:00|           12466817|       NULL|      110|
|   86790|               2|         5|        696|  

In [0]:
from pyspark.sql.functions import col, sum
missing_stats = df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns])

missing_stats.show()

+--------+----------------+----------+-----------+-----------------------+--------------------+-----------------+-------------------+-----------+---------+
|ID_Contr|ID_Tipo_Contrato|ID_Cenario|Total_Horas|Quantidade_MW_Medio_Mes|Data_Inicio_Vigencia|Data_Fim_Vigencia|ID_Contratos_Mensal|Preco_Final|ID_Agente|
+--------+----------------+----------+-----------+-----------------------+--------------------+-----------------+-------------------+-----------+---------+
|     460|               0|         0|          0|                  18511|                   0|                0|                  1|     641137|    61397|
+--------+----------------+----------+-----------+-----------------------+--------------------+-----------------+-------------------+-----------+---------+



In [0]:
df = df.drop('Preco_Final')

In [0]:
# df.where(col('Quantidade_MW_Medio_Mes') > 0).take(5)

[Row(ID_Contr=66920, ID_Tipo_Contrato=10, ID_Cenario=24, Total_Horas=744, Quantidade_MW_Medio_Mes='1822214621', Data_Inicio_Vigencia=datetime.datetime(2022, 7, 15, 0, 0), Data_Fim_Vigencia=datetime.datetime(2022, 7, 18, 0, 0), ID_Contratos_Mensal=10355788, ID_Agente=230),
 Row(ID_Contr=66920, ID_Tipo_Contrato=10, ID_Cenario=24, Total_Horas=744, Quantidade_MW_Medio_Mes='1822214621', Data_Inicio_Vigencia=datetime.datetime(2022, 2, 11, 0, 0), Data_Fim_Vigencia=datetime.datetime(2022, 2, 11, 0, 0), ID_Contratos_Mensal=9467608, ID_Agente=230),
 Row(ID_Contr=66920, ID_Tipo_Contrato=10, ID_Cenario=24, Total_Horas=744, Quantidade_MW_Medio_Mes='1822214621', Data_Inicio_Vigencia=datetime.datetime(2022, 2, 11, 0, 0), Data_Fim_Vigencia=datetime.datetime(2022, 2, 11, 0, 0), ID_Contratos_Mensal=7645089, ID_Agente=230),
 Row(ID_Contr=66920, ID_Tipo_Contrato=10, ID_Cenario=42, Total_Horas=744, Quantidade_MW_Medio_Mes='1822214621', Data_Inicio_Vigencia=datetime.datetime(2023, 3, 9, 0, 0), Data_Fim_Vige

In [0]:
df = df.drop('Quantidade_MW_Medio_Mes')

In [0]:
df = df.where(col('ID_Agente') > 0)

In [0]:
delta_table_path = "/dbfs/delta-lake-tables/silver"  
df.write.mode("overwrite").format("delta").save(delta_table_path)